Домашнее задание №2. Тема "BM25"

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [0]:
import pandas as pd
import numpy as np

In [0]:
table = pd.read_csv('quora_question_pairs_rus.csv')

In [5]:
table.head()

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1


In [0]:
N = len(table)

In [0]:
test_N = 10000

In [0]:
test_table = table.head(test_N)

In [0]:
doc_lens = []
for row in test_table['question2']:
  doc_lens.append(len(str(row).split()))

In [0]:
len_mean = np.mean(doc_lens)

In [11]:
pip install pymorphy2

In [0]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [0]:
texts = []
for row in test_table['question2']:
  texts.append(str(row))

In [0]:
import re
corpus = []#массив массивов лемм слов всех текстов
for t in texts:#for text
    ws = []
    if t != "":
      words = t.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      corpus.append(ws)

In [0]:
queries = []
for row in test_table['question1']:
  queries.append(str(row))

In [0]:
import re
query_corpus = []#массив массивов лемм слов всех текстов запросов
for q in queries:#for text
    ws = []
    if q != "":
      words = q.split()
      for w in words:
        w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()#чистим слова
        if w != "":
          p = morph.parse(w)[0]
          if p not in ws:
            ws.append(p.normal_form)
      query_corpus.append(ws)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [0]:
arrstr = []
for text in corpus:  
  s = ' '.join(text)
  arrstr.append(s)

In [0]:
X = vectorizer.fit_transform(arrstr)
matrix = X.toarray()

In [0]:
words = vectorizer.get_feature_names()

In [0]:
tf_matrix = matrix / np.array(doc_lens).reshape((-1, 1)) #матрица с обычными значениями tf

In [0]:
def tf_matrix_func(tf_matrix, b): 
    k = 2
    pairs =  np.ndenumerate(tf_matrix)
    for i, value in pairs:
        doc_id = i[0]
        l = doc_lens[doc_id]
        tf_matrix[i] = (value * (k + 1.0)) / (value + k * (1.0 - b + b * (l/len_mean)))
    return tf_matrix

In [0]:
from math import log
num_doc_qi = np.count_nonzero(matrix, axis=0) #number of docs with qi

def idf_score(word):
    word_id = words.index(word)
    num = num_doc_qi[word_id]
    score = log((test_N - num + 0.5) / (num + 0.5))
    return score

In [24]:
idf_score('кохинор')

8.804825262617976

In [0]:
all_idfs = []
for word in words:
  score = idf_score(word)
  all_idfs.append(score)

In [0]:
def tokenize(query):
  ws = []
  query_words = query.split()
  for w in query_words:
    w = re.sub('[.,-;:?!@#$%^&()_+=—\ufeff–"…«»>wwwtvsubtitlesnet]', '', w).lower()
    if w != "":
      p = morph.parse(w)[0]
      ws.append(p.normal_form)
  return ws

In [0]:
b = 0.75

In [0]:
tf_matrix_upd_1 = tf_matrix_func(tf_matrix, b) #матрица со значениями по новой формуле

In [0]:
for i in range(len(tf_matrix_upd_1)):
  for j in range(len(tf_matrix_upd_1[i])):
    tf_matrix_upd_1[i][j] = tf_matrix_upd_1[i][j] * all_idfs[j] #матрица со значениями bm25 для корпуса

Ниже функция расчета через перемножение матрицы и вектора с примером

In [0]:
def bm25_vect_ver(query):
  
  query_tok = tokenize(query)
  query_new = [' '.join(query_tok)]
  query_vector = np.array(vectorizer.transform(query_new).todense())[0]
  
  scores_bm25 = tf_matrix_upd_1.dot(query_vector)
  
  return scores_bm25

In [0]:
scores_bm25 = bm25_vect_ver('рождественские каникулы')

In [0]:
score_ids = []
for i in range(len(scores_bm25)):
  score_ids.append([i,scores_bm25[i]])

In [0]:
new_list = sorted(score_ids, key=lambda x:x[1], reverse=True)

In [35]:
for i in range(5):
  print(new_list[i])
  print(texts[new_list[i][0]])

[6587, 4.485471472199777]
каков ваш рождественский список
[7194, 0.9657913419547931]
я не хочу покупать маме рождественский подарок, что я должен делать
[0, 0.0]
что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад
[1, 0.0]
как повысить скорость интернета путем взлома через dns
[2, 0.0]
найти остаток, когда математика 23 ^ 24 математика разделена на 24 23


Функция попарного расчета

In [0]:
def bm25_pair_ver(query, b):
  k = 2
  query_tokenized = tokenize(query)
  results = []
  for i in range(test_N):
    bm25_score = 0
    for j in range(len(query_tokenized)):
      word = query_tokenized[j]
      l = doc_lens[i]
      if word in words:
        word_id = words.index(word)
        tf_val = tf_matrix[(i, word_id)]
        tf_new = (tf_val * (k + 1.0))/(tf_val + k * (1.0 - b + b * (l/len_mean)))
        bm25_score += all_idfs[word_id] * tf_new
      else:
        bm25_score += 0
    results.append(bm25_score)
  return results

Сравнение времени работы последних двух функций

In [38]:
query_0 = 'история кохинор продолжается'
from time import time
start1 = time()
result_1 = bm25_vect_ver(query_0)
time1 = str(time() - start1)
print('Time of the first function: ' + time1)

start2 = time()
result_2 = bm25_pair_ver(query_0, b)
time2 = str(time() - start2)
print('Time of the second function: ' + time2)


Time of the first function: 0.0552213191986084
Time of the second function: 3.8122305870056152


Пример топ 10 со значениями bm25

In [0]:
query_2 = 'рождественские каникулы'

In [0]:
bm25_scores = bm25_pair_ver(query_2, b)

In [0]:
new_mas = []
new_list = sorted(enumerate(bm25_scores), key=lambda x:x[1], reverse=True)
for i in new_list:
  new_mas.append(i[0])

In [42]:
top_10 = new_mas[:10]
for rank, idx in enumerate(top_10):
  print('rank: ', rank)
  print('text: ', np.array(texts)[idx])
  print('bm25_score: ', bm25_scores[idx], '\n')

rank:  0
text:  каков ваш рождественский список
bm25_score:  19.850743794745526 

rank:  1
text:  я не хочу покупать маме рождественский подарок, что я должен делать
bm25_score:  7.469915516549536 

rank:  2
text:  что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад
bm25_score:  0.0 

rank:  3
text:  как повысить скорость интернета путем взлома через dns
bm25_score:  0.0 

rank:  4
text:  найти остаток, когда математика 23 ^ 24 математика разделена на 24 23
bm25_score:  0.0 

rank:  5
text:  какая рыба выживет в соленой воде
bm25_score:  0.0 

rank:  6
text:  Я тройная луна-козерог и восхождение в козероге, что это говорит обо мне
bm25_score:  0.0 

rank:  7
text:  что делает детей активными и далеки от телефонных и видеоигр
bm25_score:  0.0 

rank:  8
text:  что я должен делать, чтобы быть великим геологом?
bm25_score:  0.0 

rank:  9
text:  когда вы используете вместо
bm25_score:  0.0 



Приверим результаты с помощью питоновкого варианта подсчета bm_25

In [43]:
pip install rank_bm25

  Created wheel for rank-bm25: filename=rank_bm25-0.2-cp36-none-any.whl size=4162 sha256=277773f944e9ba01c3b4ed2239982f3a2d01726e6d734021822299603d2bd658
  Stored in directory: /root/.cache/pip/wheels/6f/0c/1f/78945dd6a5478bbcdb50d73ac96ae5af2ffcdfcd374fd9b1bf
Successfully built rank-bm25


In [0]:
from rank_bm25 import BM25Okapi

In [0]:
bm25 = BM25Okapi(corpus)

In [0]:
query_tokenized = tokenize(query_2)

In [47]:
query_tokenized

['рождественский', 'каникулы']

In [0]:
doc_scores = bm25.get_scores(query_tokenized)

In [49]:
r = 0
for i in bm25.get_top_n(query_tokenized, corpus, n=10):
  print("rank", r)
  r += 1
  print(i)

rank 0
['каков', 'ваш', 'рождественский', 'список']
rank 1
['я', 'не', 'хотеть', 'покупать', 'мама', 'рождественский', 'подарок', 'что', 'я', 'должный', 'делать']
rank 2
['безопасно', 'ли', 'для', 'женщина', 'путешествовать', 'в', 'одиночка', 'в', 'япония']
rank 3
['какой', 'странный', 'сувенир', 'для', 'покемон', 'в', 'луч', 'солнце', 'и', 'луна']
rank 4
['быть', 'работать', 'карта', 'r', 'на', 'do', 'd', 'x']
rank 5
['сколько', 'деньга', 'быть', 'составлять', 'рупия', 'за', 'год', 'на', 'годовой']
rank 6
['как', 'увеличить', 'рост']
rank 7
['античный', 'бизнес', 'юридический', 'в', 'индия']
rank 8
['который', 'являться', 'самый', 'известный', 'поэт']
rank 9
['что', 'вызывать', 'рост', 'продажа', 'pho', 'за', 'апрель', 'год']


Попробуем посчитать точность поиска

In [0]:
def test(query, b): #функция возвращает 1, если в топ 5 есть хоть однин документ с 1 в is_duplicate, и 0, если такого нет.
  bm25_scores = bm25_pair_ver(query, b)
  mas = []
  new_list = sorted(enumerate(bm25_scores), key=lambda x:x[1], reverse=True)
  for i in new_list:
    mas.append(i[0])
  top_5 = mas[:5]
  is_dupl_scores = []
  for rank, idx in enumerate(top_5):
    is_dupl_score = np.array(test_table)[idx]
    is_dupl_scores.append(is_dupl_score[3])
  if 1 in is_dupl_scores:
    success = 1
  else:
    success = 0
  return success

Так как код работает долго, посчитаю точность только для 10 примеров

In [0]:
def acc_score(b):
  sum_score = 0
  for i in queries[:10]:
    success = test(i, b)
    sum_score += success
  acc = sum_score/len(queries[:10])
  return acc

In [0]:
acc_1 = acc_score(0.75)

In [80]:
acc_1

0.8

In [0]:
acc_2 = acc_score(0)

In [82]:
acc_2

0.9

In [0]:
acc_3 = acc_score(1)

In [84]:
acc_3

0.8

Видно, что на маленькой выборке результаты получились почти одинаковые.